# Abstract Data Types

Nous commencons par créer une classe Sequence, qui est l'ADT qui représente une séquence d’acides aminés et toutes les opérations qu’on  peut  exécuter  sur  une  séquence (exemple:  retourner  un  acide  aminé  à  une  certaine  position ou visualiser la séquence en format FASTA).

In [1]:
class Sequence:
    def __init__(self, acids, annotation=None):
        self.acids = acids
        self.annotation = annotation
        
    def __getitem__(self, position):
        """ Renvoie l'acide aminé à la position donnée. Syntaxe: sequence[position] """
        return self.acids[position]
    
    def toFASTAFormat(self):
        return ">" + self.annotation + "\n" + self.acids
    
    def __repr__(self):
        return "Annotation: %s \nAcids: %s" % (self.annotation, self.acids)
    
    def length(self):
        return len(self.acids)
        

Nous créeons aussi une classe Score, qui est un ADT qui représente une matrice de substitution et les opérations qu'on peut exécuter sur cette matrice.

In [2]:
class Score:
    def __init__(self, matrix):
        self.matrix = matrix
    
    def getNumLetters(self):
        return len(self.matrix)
    
    def getLetters(self):
        return self.matrix.keys()

    def getCost(self, a, b):
        return self.matrix[a.upper()][b.upper()]

## Parser

Nous devons créer des objets des classes Sequence et Score à partir des fichiers fournis. Il faut donc créer un parser pour les fichier des séquences (FASTA) et pour les fichiers avec les matrices de substitution (BLOSUM et PAM).

## FASTA

Les séquences d'acides aminés sont codées dans le format FASTA. Ce format est très simple.
Chaque séquence est encodée par une annotation et ensuite la liste d'acides aminés. Voici un exemple:

>&gt;sp|Q13526|PIN1_HUMAN Peptidyl-prolyl cis-trans isomerase NIMA-interacting 1 OS=Homo sapiens GN=PIN1 PE=1 SV=1
MADEEKLPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSGGKNGQGEPARVRCSHL
LVKHSQSRRPSSWRQEKITRTKEEALELINGYIQKIKSGEEDFESLASQFSDCSSAKARG
DLGAFSRGQMQKPFEDASFALRTGEMSGPVFTDSGIHIILRTE

La prèmiere ligne (qui commence par >) est une annotation contenant des informations sur la séquence. Ensuite viennent une série de lignes contenant la séquence d'acides aminés (avec un maximum de 80 caractères par ligne, en général, pour faciliter la lisibilité). 

Un fichier FASTA est constitué d'une série de séquences formattées de cette façon.

In [3]:
def parseFASTASequencesFromFile(filename):
    with open(filename, 'r') as f:
        return parseFASTASequences(f)

def parseFASTASequences(lines):
    sequences = []
    annotation, acids = "", ""
    for line in lines:
        line = line.strip()

        justEndedPreviousSequence = (len(line) == 0 or line.startswith(">")) and acids != ""
        if justEndedPreviousSequence:
            sequences.append(Sequence(acids, annotation))

        if line.startswith(">"):
            annotation = line[1:]
            acids = ""
        else:
            acids += line

    return sequences


Appliquons ces parsers aux fichiers donnés:

In [4]:
sequencesFilenames = ['protein-sequences.fasta', 'WW-sequence.fasta']
proteinSequences, wwSequences = [parseFASTASequencesFromFile(filename) for filename in sequencesFilenames]

sequence = proteinSequences[0]

Faisons quelques tests pour vérifier notre fonction:

In [5]:
print("La première sequence est: \n")
print(sequence)

La première sequence est: 

Annotation: sp|P46937|YAP1_HUMAN Transcriptional coactivator YAP1 OS=Homo sapiens GN=YAP1 PE=1 SV=2 
Acids: MDPGQQPPPQPAPQGQGQPPSQPPQGQGPPSGPGQPAPAATQAAPQAPPAGHQIVHVRGDSETDLEALFNAVMNPKTANVPQTVPMRLRKLPDSFFKPPEPKSHSRQASTDAGTAGALTPQHVRAHSSPASLQLGAVSPGTLTPTGVVSGPAATPTAQHLRQSSFEIPDDVPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRKAMLSQMNVTAPTSPPVQQNMMNSASGPLPDGWEQAMTQDGEIYYINHKNKTTSWLDPRLDPRFAMNQRISQSAPVKQPPPLAPQSPQGGVMGGSNSNQQQQMRLQQLQMEKERLRLKQQELLRQAMRNINPSTANSPKCQELALRSQLPTLEQDGGTQNPVSSPGMSQELRTMTTNSSDPFLNSGTYHSRDESTDSGLSMSSYSVPRTPDDFLNSVDEMDTGDTINQSTLPSQQNRFPDYLEAIPGTNVDLGTLEGDGMNIEGEELMPSLQEALSSDILNDMESVLAATKLDKESFLTWL


In [6]:
print("\nLe premier acide aminé de la séquence est: " + sequence[0])


Le premier acide aminé de la séquence est: M


### Matrices de substitution

Les matrices de substitution sont aussi dans un format simple et intuitif. Le fichier commence par quelques lignes de commentaires (commencent par #) et puis est suivi de la grille des coûts de substitution.

Voici un exemple simplifié:

>```
# Ma matrice
# Il y 5 acides aminés dans cet exemple
   A  R  N  D  C
A  4 -1 -2 -2  0 -
R -1  5  0 -2 -3  
N -2  0  6  1 -3  
D -2 -2  1  6 -3  
C  0 -3 -3 -3  9 
```

In [7]:
def parseSubstitutionMatrixFromFile(filename):
    with open(filename, 'r') as f:
        return parseSubstitutionMatrix(f)

def parseSubstitutionMatrix(lines):
    matrix = {}
    
    for line in lines:
        if line.startswith('#') or line.strip() == "": # Skip comments
            continue
        elif line.startswith('  '): # First line with letters
            letters = line.strip().split()
        else:
            letter, *costs = line.strip().split()
            matrix[letter] = {}
            for (i, cost) in enumerate(costs):
                matrix[letter][letters[i]] = int(cost)
    
    return Score(matrix)


Essayons d'appliquer ce parser à nos fichiers:

In [8]:
matricesFilenames = ['blosum62.txt', 'blosum80.txt', 'pam60.txt', 'pam120.txt']
blosum62, blosum80, pam60, pam120 = [parseSubstitutionMatrixFromFile(filename) for filename in matricesFilenames]

print("\nLe coût de subsition de R et C dans Blosum62 devrait être -3 et est %d" % blosum62.getCost('R', 'C'))
letters = blosum62.getLetters()
print("Il y a %d colonnes dans Blosum62: %s" % (len(letters), ''.join(sorted(letters))))


Le coût de subsition de R et C dans Blosum62 devrait être -3 et est -3
Il y a 24 colonnes dans Blosum62: *ABCDEFGHIKLMNPQRSTVWXYZ


# Alignement global 

Maintenant qu'on a défini ces ADT, on peut commencer à implementer les algorithmes d'alignement. Pour l'alignement global, nous allons implémenter l'algorithme de *Needleman-Wunsch* avec une penalité affine.

Lorsque la penalité est affine, cela vaut dire que commencer une séquence d'espaces est plus couteux que de la prolonger. Par exemple, le 1er espace peut avoir un cout de 10 alors que les espaces suivent n'ont qu'un cout de 2.

In [9]:
def printMatrix(matrix):
    for line in matrix:
        print(line)

def makeMatrix(width, height, val=0):
    if type(val) in (int, str):
        return [[val] * width for _ in range(height)]
    elif val == []:
        return [[[] for _ in range(width)]for _ in range(height)]
    else:
        return [[val for _ in range(width)]for _ in range(height)]

def getValue(matrix, x, y):
    if x < 0 or y < 0:
        return float('-inf')
    else:
        return matrix[y][x]

def getSolutionsFromArrowGridHelper(previous, k, x, y):
    # TODO: write a non recursive version of this function
    if x == 0 and y == 0:
        return [[(0, 0)]]
    
    paths = []
    solutionsFound = 0
    for newX, newY in previous[y][x]:
        if k <= solutionsFound:
            break
        
        sols = getSolutionsFromArrowGridHelper(previous, k - solutionsFound, newX, newY)
        solutionsFound += len(sols)
        for path in sols:
            paths.append([(x, y)] + path)
        
    return paths

def getSolutionsFromArrowGridHelperIterative(previous, k, x, y, isGlobal=True):
    completePaths = []
    incompletePaths = [[(x, y)]]
    numPaths = 1
    
    while len(incompletePaths) > 0 and len(completePaths) <= k:
        path = incompletePaths.pop()
        lastX, lastY = path[-1]
            
        for (i, previousCoords) in enumerate(previous[lastY][lastX]):
            if i > 0: # 2+ paths from this cell
                numPaths += 1
                if numPaths > k: # No need to consider new paths if we already have enough
                    break
            
            if not isGlobal and previousCoords is None:
                completePaths.append(path)
                continue
                
            newX, newY = previousCoords
            isComplete = (newX == 0 and newY == 0)
            
            if isComplete:
                completePaths.append(path + [(newX, newY)])
            else:
                incompletePaths.append(path + [(newX, newY)])

    return completePaths

def getSolutionsFromArrowGrid(previous, k, x=None, y=None, isGlobal=True):
    if x is None or y is None:
        x, y = len(previous[0]) - 1, len(previous) - 1
        
    return [path[::-1] for path in getSolutionsFromArrowGridHelperIterative(previous, k, x, y, isGlobal)]        

            
def getBestGlobalAlignements(sequence1, sequence2, score, startGap=4, keepGap=1, k=1):
    """
    k: the maximum number of alignement returned (note: less than k alignements may be returned)
    """
    height, width = sequence1.length(), sequence2.length()
    v, w, s = [makeMatrix(width + 1, height + 1) for _ in range(3)]
    previous = makeMatrix(width + 1, height + 1, [])
    
    for i in range(1, width + 1):
        s[0][i] = -(startGap + (i - 1) * keepGap)
        v[0][i] = float('-inf')
        previous[0][i] = [(i - 1, 0)]
    
    
    for i in range(1, height + 1):
        s[i][0] = -(startGap + (i - 1) * keepGap)
        w[i][0] = float('-inf')
        previous[i][0] = [(0, i - 1)]
    
    
    
    for x in range(1, width + 1):
        for y in range(1, height + 1):
            v[y][x] = max(s[y - 1][x] - startGap, v[y - 1][x] - keepGap)
            w[y][x] = max(s[y][x - 1] - startGap, w[y][x - 1] - keepGap)
            replacementCost = score.getCost(sequence1[y - 1], sequence2[x - 1])

            costsMap = {
                (x, y - 1): v[y][x],
                (x - 1, y): w[y][x],
                (x - 1, y - 1): s[y - 1][x - 1] + replacementCost
            }

            s[y][x] = max(costsMap.values())

            for coord, cost in costsMap.items():
                if cost == s[y][x]:
                    previous[y][x].append(coord)

    score = s[height][width]
    
    return s, getSolutionsFromArrowGrid(previous, k), score, previous

In [10]:
from pprint import pprint
def showPath(path):
    x, y = path[-1]
    mat = makeMatrix(x + 1, y + 1, '*')
    for x, y in path:
        mat[y][x] = 'X'
   
    for line in mat:
        print(' '.join(line))
    print()

a = Sequence('writers')
b = Sequence('vintner')

scoreMatrix, paths, score, arrowMatrix = getBestGlobalAlignements(a, b, blosum62, startGap=10, keepGap=3, k=100)
print("There are %d paths\n" % len(paths))
for path in paths:
    showPath(path)
    print(path)
    
print("The score matrix is\n")
printMatrix(scoreMatrix)

print("\nThe arrow matrix is\n")
printMatrix(arrowMatrix)

There are 1 paths

X * * * * * * *
* X * * * * * *
* * X * * * * *
* * * X * * * *
* * * * X * * *
* * * * * X * *
* * * * * * X *
* * * * * * * X

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7)]
The score matrix is

[0, -10, -13, -16, -19, -22, -25, -28]
[-10, -3, -13, -16, -18, -22, -25, -28]
[-13, -13, -6, -13, -17, -18, -22, -20]
[-16, -10, -9, -9, -14, -20, -21, -25]
[-19, -16, -11, -9, -4, -14, -17, -20]
[-22, -21, -19, -11, -10, -4, -9, -17]
[-25, -25, -24, -19, -12, -10, -4, -4]
[-28, -27, -27, -23, -18, -11, -10, -5]

The arrow matrix is

[[], [(0, 0)], [(1, 0)], [(2, 0)], [(3, 0)], [(4, 0)], [(5, 0)], [(6, 0)]]
[[(0, 0)], [(0, 0)], [(1, 0), (1, 1)], [(2, 1)], [(3, 0)], [(4, 1)], [(5, 1), (5, 0)], [(6, 1), (6, 0)]]
[[(0, 1)], [(0, 1), (1, 1)], [(1, 1)], [(2, 1)], [(3, 1)], [(4, 1)], [(5, 1)], [(6, 1)]]
[[(0, 2)], [(0, 2)], [(1, 2)], [(2, 2)], [(3, 2)], [(4, 2)], [(5, 2)], [(6, 2)]]
[[(0, 3)], [(0, 3)], [(1, 3)], [(2, 3)], [(3, 3)], [(4, 4), (4, 3)], [(5, 4)], 

From this, we can reconstruct the best alignements between these two sequences:

In [11]:
def showAlignments(path, sequence1, sequence2):
    text1, text2 = "", ""
    prevX, prevY = path[0]
    
    for (x, y) in path[1:]:
        #if x == 0 and y == 0:
        #    continue
        
        if x == prevX and y == prevY + 1: # Down
            text2 += "_"
            text1 += sequence1[x - 1] # Check this
        elif x == prevX + 1 and y == prevY: # Right
            text1 += "_"
            text2 += sequence2[y - 1] # Check this
        elif x == prevX + 1 and y == prevY + 1: # Diagonal
            text1 += sequence1[x - 1]
            text2 += sequence2[y - 1]
        
        prevX, prevY = x, y
    
    return text1, text2

def printAlignementResult(seq1, seq2, result, showScore = True, showLength = False):
    scoreMatrix, paths, score, arrowMatrix = result
    if showScore:
        print("The alignements score of these sequences is %d\n" % score)
        
    print(seq1, end="\n\n")
    print(seq2, end="\n\n")
    print("Alignements found: %d (may be artificially limited)\n" % len(paths))
    for path in paths:
        text1, text2 = showAlignments(path, seq1, seq2)
        if showLength:
            print("The length of the alignemnt is %d" % len(text1))
            
        print(text1)
        print(text2)
        print()

def compareSequences(seq1, seq2, matrix=blosum62, startGap=4, keepGap=1, k=5, showScore=True):
    scoreMatrix, *rest = result = getBestGlobalAlignements(seq1, seq2, matrix, startGap, keepGap, k)
    printAlignementResult(seq1, seq2, result, showScore)
        
ww1, ww2 = wwSequences[0], wwSequences[1]
compareSequences(ww1, ww2)

The alignements score of these sequences is 77

Annotation: sp|P46937|171-204 
Acids: VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK

Annotation: sp|P46934|610-643 
Acids: SPLPPGWEERQDILGRTYYVNHESRRTQWKRPTP

Alignements found: 1 (may be artificially limited)

VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK
SPLPPGWEERQDILGRTYYVNHESRRTQWKRPTP



On peut regarder quelle paires de séquences du fichier *ww-sequences.fasta* ont des alignements avec le score maximal.

In [12]:
import itertools

def getBestMatches(sequences):
    bestScore = float('-inf')
    bestResults = {}
    
    for seq1, seq2 in itertools.combinations(sequences, 2):
        scoreMatrix, paths, score, arrowMatrix = result = getBestGlobalAlignements(seq1, seq2, blosum62, startGap=4, keepGap=1, k=1)
        if score > bestScore:
            bestScore = score
            bestResults = {(seq1, seq2): result}
        elif score == bestScore:
            bestResults[(seq1, seq2)] = result
    
    return bestResults, bestScore

# TODO: improve perfomance. Right now it's dumb as a rock
bestResults, score = getBestMatches(wwSequences)
print("The best matches have a score of %d" % score)
print("They are between these sequences: ")
i = 1
for (seq1, seq2), result in bestResults.items():
    print("\n------------- Sequences %d -----------------\n" % i)
    printAlignementResult(seq1, seq2, result, showScore=False)
    i += 1

The best matches have a score of 117
They are between these sequences: 

------------- Sequences 1 -----------------

Annotation: sp|P46934|610-643 
Acids: SPLPPGWEERQDILGRTYYVNHESRRTQWKRPTP

Annotation: sp|P46934|892-925 
Acids: GPLPPGWEERTHTDGRIFYINHNIKRTQWEDPRL

Alignements found: 1 (may be artificially limited)

SPLPPGWEER__ILLLGRTYYVNHESRRTQWKRPTP
GPLPPGWEERRRTH__GRIFYINHNIKRTQWEDPRL



# Alignement local

Pour calculer l'alignement local entre séquences, on implémente l'algorithme de Smith-Waterman. Celui ci est extremement proche de l'algorithme de Needleman-Wunsch. Voici les différences:

![Alignement local](images/local.png)

In [13]:
def findMaximumsInMatrix(matrix):
    maximums = []
    maxVal = float('-inf')
    for (y, line) in enumerate(matrix):
        for (x, value) in enumerate(line):
            if value > maxVal:
                maxVal = value
                maximums = [(x, y)]
            elif value == maxVal:
                maximums.append((x, y))
    return maximums
    

def getBestLocalAlignements(sequence1, sequence2, score, startGap=4, keepGap=1, l=1):
    """
    l: the maximum number of alignement returned (note: less than l alignements may be returned)
    """
    height, width = sequence1.length(), sequence2.length()
    v, w, s = [makeMatrix(width + 1, height + 1) for _ in range(3)]
    previous = makeMatrix(width + 1, height + 1, [])
    
#     for i in range(1, width + 1):
#         s[0][i] = -(startGap + (i - 1) * keepGap)
#         previous[0][i] = [(i - 1, 0)]
    
    
#     for i in range(1, height + 1):
#         s[i][0] = -(startGap + (i - 1) * keepGap)
#         previous[i][0] = [(0, i - 1)]
    
    
    for x in range(1, width + 1):
        for y in range(1, height + 1):
            v[y][x] = max(s[y - 1][x] - startGap, v[y - 1][x] - keepGap)
            w[y][x] = max(s[y][x - 1] - startGap, w[y][x - 1] - keepGap)
            replacementCost = score.getCost(sequence1[y - 1], sequence2[x - 1])

            costsMap = {
                (x, y - 1): v[y][x],
                (x - 1, y): w[y][x],
                (x - 1, y - 1): s[y - 1][x - 1] + replacementCost,
                None: 0
            }

            s[y][x] = bestScore = max(costsMap.values())
        
            for coord, cost in costsMap.items():
                if cost == bestScore:
                    previous[y][x].append(coord)

    
    
    bestCoords = findMaximumsInMatrix(s)
    x, y = bestCoords[0]
    score = s[y][x]
    
    paths = []
    maxPathsLeft = l
    for (x, y) in bestCoords:
        newPaths = getSolutionsFromArrowGrid(previous, maxPathsLeft, x, y, isGlobal=False)
        
        paths.extend(newPaths)
        maxPathsLeft -= len(newPaths)
        if maxPathsLeft <= 0:
            break
        
    return s, paths, score, previous

Observons les résultats de cette fonctions lorsqu'on utilise les données du fichier *protein-sequences.fasta*.

In [14]:
result = getBestLocalAlignements(ww1, ww2, blosum62, startGap=4, keepGap=1, l=1)
printAlignementResult(ww1, ww2, result, showScore = True, showLength = True)

scoreMatrix, paths, *rest = result
print(paths)

The alignements score of these sequences is 81

Annotation: sp|P46937|171-204 
Acids: VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK

Annotation: sp|P46934|610-643 
Acids: SPLPPGWEERQDILGRTYYVNHESRRTQWKRPTP

Alignements found: 1 (may be artificially limited)

The length of the alignemnt is 31
PLPAGWEMAKTSSGQRYFLNHIDQTTTWQDP
PLPPGWEERQDILGRTYYVNHESRRTQWKRP

[[(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 21), (22, 22), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 31), (32, 32)]]
